In [2]:
import xarray as xr
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

path = '/home/akazemi3/Desktop/MB_Lab_Project/'
sys.path.append(path)
from analysis.neural_data_regression.tools.regression import *

from tools.processing import *
from tools.loading import *

import pandas as pd
import seaborn as sns
import pickle
import tables
import numpy as np

In [3]:
def get_best_alpha(data_dict,alphas,legend_name_dict=None):
    

    root_path = '/data/atlas/model_scores'
    

    df = pd.DataFrame()
    
    
    index = 0
    for model, regions in data_dict.items():
                
        for a in alphas:

            regression = f'Ridge(alpha={a})' 
                
            for region in regions:

                data = xr.open_dataset(os.path.join(root_path,f'{model}_{regression}'))
                r_values = data.where(data.region == region,drop=True).r_value.values
                mean_r = np.mean(r_values)
                df_tmp =  pd.DataFrame({'mean_score':mean_r,
                                        'alpha':a,
                                        'model':model,
                                        'region':region},index=[index])
                df = pd.concat([df,df_tmp])
                index += 1

    
    df = df.groupby(['region','model','alpha']).mean().reset_index()
    if legend_name_dict is not None:
        df.model = df.model.map(legend_name_dict)
    max_idx = df.groupby(['model','region']).agg({'mean_score':('idxmax')}).reset_index()['mean_score'].tolist()
    df_max_alphas = df.loc[max_idx].reset_index(drop=True)

    return df_max_alphas
    

In [35]:
ds = xr.open_dataarray('/data/atlas/activations/model_final_mp_naturalscenes_zscored_processed')

In [40]:
ds = ds.set_index({'presentation':'stimulus_id'})

In [48]:
test = ds.sel(presentation=['image00008', 'image00005', 'image00002'])

In [49]:
test = test.sortby('presentation', ascending=True)

In [53]:
torch.Tensor(test.values.squeeze())

tensor([[ 7.4778,  5.8183,  6.2178,  ...,  5.8474,  6.0099,  7.4587],
        [ 7.8772, 10.4161,  7.4834,  ...,  7.1513,  9.3592,  6.1329],
        [10.3289, 10.5337,  8.5822,  ..., 11.5284, 10.8469, 10.5990]])

In [54]:
torch.Tensor(test.values)

tensor([[ 7.4778,  5.8183,  6.2178,  ...,  5.8474,  6.0099,  7.4587],
        [ 7.8772, 10.4161,  7.4834,  ...,  7.1513,  9.3592,  6.1329],
        [10.3289, 10.5337,  8.5822,  ..., 11.5284, 10.8469, 10.5990]])

In [4]:

# # get best alpha using the model_scores
# activations_identifier = model_name + '_' + dataset
# alphas =  [10**a for a in range(5,7)] 
# data_dict = {activations_identifier:regions}
# df_best_alpha = get_best_alpha(data_dict,alphas)
# alpha = df_best_alpha.alpha[0]

In [7]:
model_name = 'model_final_mp_shared'
dataset= 'naturalscenes_zscored_processed'
regions = ['V4']
activations_identifier = model_name + '_' + dataset

alpha = 1000000

scores_identifier = activations_identifier + '_'  + f'Ridge(alpha={alpha})' 
path_to_betas = f'/data/atlas/regression_betas/{scores_identifier}'


l = []
n_folds = len(os.listdir(path_to_betas))

for i in range(n_folds):
    with open(f'{path_to_betas}/betas_fold_{i}','rb') as fp:
        l.append(pickle.load(fp))
    
 
positive_mean_betas = np.abs(sum(l)/len(l))
# mean_channel_betas = positive_mean_betas.mean(axis=0)
# p = np.percentile(mean_channel_betas,90)
# idx = np.argwhere(mean_channel_betas > p)
# idx = idx.reshape(-1)

In [ ]:
sns.violinplot(positive_mean_betas)

In [ ]:
filename= '/home/akazemi3/Desktop/MB_Lab_Project/models/all_models/core_activations/model_final_mp_2l_naturalscenes_zscored_processed.h5'
f = tables.open_file(filename, mode='r')
selected_channels = f.root.data[:,idx,:,:]

In [2]:
xr.open_dataset('/data/atlas/activations/model_final_mp_2l_naturalscenes_zscored_processed')

<xarray.Dataset>
Dimensions:      (features: 20000, presentation: 73000)
Coordinates:
    stimulus_id  (presentation) object ...
Dimensions without coordinates: features, presentation
Data variables:
    x            (presentation, features) float32 ...

In [17]:
selected_channels.shape

(73000, 2000, 2, 2)